In [1]:
from piper import piper
from piper.verbs import *

piper version 0.0.7, last run: Friday, 26 February 2021 15:21:33


In [2]:
def cleaning_function(df):
    
    df.loc[df.qty.str.match(r'[\w]') == True, 'qty'] = (df.qty.str.strip()
                                                          .str.replace(r'[\s\-]', '', regex=True)
                                                          .str.replace(',(\d{2})$',r'.\1', regex=True)
                                                          .str.replace(r',', '', regex=True))
    df.loc[df.price.str.match(r'[\w]') == True, 'price'] = (df.price
                                                              .str.replace(r'\s+', r'', regex=True)
                                                              .str.replace(r'\.{2}',r'', regex=True))
    
    df.loc[df.price.str.match(r'[\W]') == True, 'price'] = (df.price
                                                              .str.replace(r',', r'.', regex=True))
 
    df.loc[df.price.str.match(r'[\w]') == True, 'price'] = (df.price
                                                              .str.replace(r',', r'.', regex=True)
                                                              .str.replace(r'\s+', r'', regex=True)
                                                              .str.replace(r'\.{2}',r'', regex=True))

    df.loc[df.price.str.match(r'[\w]') == True, 'price'] = (df.price     
                                                              .str.replace(r'\.{2}',r'', regex=True)
                                                              .str.replace(r'(\.\d)\.',r'\1', regex=True))
    
    df['description'] = (df['description'].str.lower()
                                      .str.strip()
                                      .str.title()
                                      .str.replace('ri','ir', regex=True)
                                      .str.replace(r'(\w)\s+(\w)', r'\1 \2', regex=True))
    
    df.second_col = (df.second_col
                   .str.strip()
                   .str.title()
                   .str.replace(r'(\w+),\s+(\w+)', r'\2 \1', regex=True)
                   )
    
    df.price = df.price.astype(float).round(2)
    df.qty = pd.to_numeric(df.qty).astype(int)
    
    df.effective = pd.to_datetime(df.effective)
    df.expired = pd.to_datetime(df.expired)
    df['duration'] = (df.expired - df.effective).dt.days
    df.drop(columns=['date'], inplace=True)
    
    expanded_list = df.delimitted_list.str.split(';', n=2, expand=True)
    expanded_list.columns = ['list_1', 'list_2', 'list_3']
    df = pd.concat([df, expanded_list], axis=1)

    return df

In [3]:
columns_renamed = ['grouping', 'orders', 'description', 'second_col',
                   'qty', 'price', 'effective', 'expired', 'date', 'delimitted_list']

In [4]:
%%piper 
df <- pd.read_excel('inputs/Test excel workbook.xlsx')
>> set_columns(columns_renamed)
>> cleaning_function() 
>> relocate('duration', loc='after', ref_column='expired')
>> assign(sales = lambda x : x.qty * x.price)
>> relocate('sales', loc='after', ref_column='price')
>> order_by(by=['second_col', 'qty']) 

In [5]:
%piper df >> head()

12 rows, 14 columns


,grouping,orders,description,second_col,qty,price,sales,effective,expired,duration,delimitted_list,list_1,list_2,list_3
0,A100,23899001,First Row,Aidan Scally,14,1.23,17.22,2015-10-21,2019-12-31,1532,123;456;789,123,456,789
3,A102,23899004,Fourth Row,Alan Denton,13,49.00,637.00,2018-10-21,2022-12-31,1532,cat;dog;books,cat,dog,books
10,A102,23899011,Fourth Row,Alan Denton,13,49.00,637.00,2018-10-21,2022-12-31,1532,d20;tg54;kde40,d20,tg54,kde40
8,A105,23899009,Ninth Row,Carsten Seiffert,24,98.00,2352.00,2018-05-21,2025-12-10,2760,1304;53-02;6932,1304,53-02,6932


In [6]:
list_of_groups = ['A100', 'A101', 'A103']
duration_criteria = 1600

query = """(grouping in @list_of_groups) and\
           (~second_col.str.startswith('Theo')) or\
           (duration >= @duration_criteria) 
        """

In [7]:
%%piper

# df.query(query).head(2)
df >> where(query, global_dict=globals()) >> head()

8 rows, 14 columns


,grouping,orders,description,second_col,qty,price,sales,effective,expired,duration,delimitted_list,list_1,list_2,list_3
0,A100,23899001,First Row,Aidan Scally,14,1.23,17.22,2015-10-21,2019-12-31,1532,123;456;789,123,456,789
8,A105,23899009,Ninth Row,Carsten Seiffert,24,98.00,2352.00,2018-05-21,2025-12-10,2760,1304;53-02;6932,1304,53-02,6932
9,A105,23899010,Tenth Row,Craig Freer,1800,563.00,1013400.00,2019-10-21,2025-12-31,2263,40a3;64g;023;,40a3,64g,023;
5,A103,23899006,Sixth Row,Derek Hunt,532,634.23,337410.36,2019-04-21,2020-05-07,382,alpha; bravo; charlie,alpha,bravo,charlie


In [8]:
piped_query = f"""(grouping in {list_of_groups}) and\
                 (~second_col.str.startswith('Theo')) or\
                 (duration >= {duration_criteria}) """

%piper df >> where(piped_query) >> head()

8 rows, 14 columns


,grouping,orders,description,second_col,qty,price,sales,effective,expired,duration,delimitted_list,list_1,list_2,list_3
0,A100,23899001,First Row,Aidan Scally,14,1.23,17.22,2015-10-21,2019-12-31,1532,123;456;789,123,456,789
8,A105,23899009,Ninth Row,Carsten Seiffert,24,98.00,2352.00,2018-05-21,2025-12-10,2760,1304;53-02;6932,1304,53-02,6932
9,A105,23899010,Tenth Row,Craig Freer,1800,563.00,1013400.00,2019-10-21,2025-12-31,2263,40a3;64g;023;,40a3,64g,023;
5,A103,23899006,Sixth Row,Derek Hunt,532,634.23,337410.36,2019-04-21,2020-05-07,382,alpha; bravo; charlie,alpha,bravo,charlie


In [9]:
def pipe(df, *args, **kwargs):
    return df

In [10]:
%%piper -i
df >> count('effective') >> pipe(pipe=True) >> head()

7 rows, 2 columns


(df
.pipe(count, 'effective')
.pipe(pipe, pipe=True)
.pipe(head))


,effective,n
0,2019-10-21,5
1,2018-10-21,2
2,2019-04-21,1
3,2017-10-21,1
